# Importing Python Packages

In [1]:
import openpyxl
import xlrd
import os
import glob

from pandas import DataFrame as df
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean

In [2]:
# import python packages

# import nltk (natural language tool kit)
import nltk
# stopwords, FreqDist, word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import requests
from lxml import html
import csv
import pandas as pd

from os import path

import matplotlib.pyplot as plt
%matplotlib inline


#regular expression
import re
#csv
import csv
# Excel-like format
import pandas as pd
#data visualization
import matplotlib.pyplot as plt
% matplotlib inline

import numpy as np

import statsmodels.api as sm

import statsmodels.formula.api as smf

# Importing Data

## Opening the Correct Directory

In [3]:
os.chdir('C:\\Users\\millsj.ADMIN\\Desktop\\GENBA 894\\2018_DataChallenges_Teradata\\Participants\\')

In [32]:
# filenames
excel_names = ["2013-2017 Bike MS Participants.xlsx"]

# read them in
excels = [pd.ExcelFile(name) for name in excel_names]

# turn them into dataframes
frames = [x.parse(x.sheet_names[0],index_col=None) for x in excels]

# delete the first row for all frames except the first
# i.e. remove the header row -- assumes it's the first
frames[1:] = [df[1:] for df in frames[1:]]

# concatenate them..
participants = pd.concat(frames)

# Check headers
participants.head(3)

,Security Category Name,Fiscal Year,Internal Event Name,Event Date,Participation Type Name,Team Name,Team Creation Date,Team Division,Team ID,Contact ID,...,Address - Participant City,Address - Participant ZIP/Postal Code,Registration Type,Event ID,Participant Gender,Participant Goal($),Suggested Participant Goal($),Source Code Type,Source Code Text,Sub Source Code Text
0,CAL Bike Events,2017,CAL 2017 Bike MS: Pasadena,2017-03-25 15:06:00,Cyclist,Team Left Hand,2016-09-28 09:37:00,Corporate,495470.0,4639884,...,Louisville,80027,Online,28217,Female,2000.0,900.0,6,www.facebook.com,/
1,COC Bike Events,2015,COC 2015 Bike MS: Colorado,2015-06-27 13:42:00,Two-Day Participant,Team Left Hand,2014-11-17 09:13:00,Corporate,407399.0,4639884,...,Louisville,80027,Offline,25572,Female,5000.0,1075.0,NaN,NaN,NaN
2,COC Bike Events,2016,COC 2016 Bike MS: Colorado,2016-06-25 11:08:00,Two Day Cyclist,Team Left Hand,2015-12-02 16:43:00,Corporate,457408.0,4639884,...,Louisville,80027,Online,27037,Female,5000.0,1095.0,NaN,NaN,NaN


In [33]:
# convert string to integers
#biketeams['Team Captain Accept Email'] = (biketeams['Team Captain Accept Email'] == 'TRUE').astype(int)

In [55]:
# make sure the conversion worked
#biketeams.head(3)

# Looking for Data Anomalies and Cleaning Addresses

In [56]:
# List of all headers
# participants[['Security Category Name','Fiscal Year','Internal Event Name','Event Date','Participation Type Name','Team Name','Team Creation Date','Team Division','Team ID','Contact ID','Member ID','Participant Accept Email','Registration Date','Registration Active Status','Is Team Captain','Is Secondary Registration','Is Prior Participant','Emails Sent','Total of All Confirmed Gifts($)','Total From Participant($)','Total Not From Participant($)','Number From Participant','Number Not From Participant','Participant Email Status','Participant Employer','Participant Occupation','Participant Connection to MS','Address  -  Participant State/Province','Address  -  Participant County','Address  -  Participant City','Address -  Participant ZIP/Postal Code','Registration Type','Event ID','Participant Gender','Participant Goal($)','Suggested Participant Goal($)','Source Code Type','Source Code Text','Sub Source Code Text']]

In [34]:
# counting zero values
print((participants[['Security Category Name','Fiscal Year','Internal Event Name','Event Date','Participation Type Name','Team Name','Team Creation Date','Team Division','Team ID','Contact ID','Member ID','Participant Accept Email','Registration Date','Registration Active Status','Is Team Captain','Is Secondary Registration','Is Prior Participant','Emails Sent','Total of All Confirmed Gifts($)','Total From Participant($)','Total Not From Participant($)','Number From Participant','Number Not From Participant','Participant Email Status','Participant Employer','Participant Occupation','Participant Connection to MS','Address  -  Participant State/Province','Address  -  Participant County','Address  -  Participant City','Address -  Participant ZIP/Postal Code','Registration Type','Event ID','Participant Gender','Participant Goal($)','Suggested Participant Goal($)','Source Code Type','Source Code Text','Sub Source Code Text']] == 0).sum())


Security Category Name                        0
Fiscal Year                                   0
Internal Event Name                           0
Event Date                                    0
Participation Type Name                       0
Team Name                                     0
Team Creation Date                            0
Team Division                                 0
Team ID                                       0
Contact ID                                    0
Member ID                                     0
Participant Accept Email                  21551
Registration Date                             0
Registration Active Status                    0
Is Team Captain                           59304
Is Secondary Registration                 65051
Is Prior Participant                          0
Emails Sent                               40802
Total of All Confirmed Gifts($)           10324
Total From Participant($)                 30001
Total Not From Participant($)           

In [35]:
# Counting missing values
participantsnull=(participants[['Security Category Name','Fiscal Year','Internal Event Name','Event Date','Participation Type Name','Team Name','Team Creation Date','Team Division','Team ID','Contact ID','Member ID','Participant Accept Email','Registration Date','Registration Active Status','Is Team Captain','Is Secondary Registration','Is Prior Participant','Emails Sent','Total of All Confirmed Gifts($)','Total From Participant($)','Total Not From Participant($)','Number From Participant','Number Not From Participant','Participant Email Status','Participant Employer','Participant Occupation','Participant Connection to MS','Address  -  Participant State/Province','Address  -  Participant County','Address  -  Participant City','Address -  Participant ZIP/Postal Code','Registration Type','Event ID','Participant Gender','Participant Goal($)','Suggested Participant Goal($)','Source Code Type','Source Code Text','Sub Source Code Text']])

print(participantsnull.isnull().sum())

Security Category Name                        0
Fiscal Year                                   0
Internal Event Name                           0
Event Date                                    0
Participation Type Name                       0
Team Name                                 11509
Team Creation Date                        11509
Team Division                             12209
Team ID                                   11509
Contact ID                                    0
Member ID                                    74
Participant Accept Email                      0
Registration Date                             0
Registration Active Status                    0
Is Team Captain                               0
Is Secondary Registration                     0
Is Prior Participant                          0
Emails Sent                                   0
Total of All Confirmed Gifts($)               0
Total From Participant($)                     0
Total Not From Participant($)           

In [36]:
#compute count of values in columns
participants['Registration Type'].value_counts()


Online     57666
Offline     7833
Name: Registration Type, dtype: int64

In [37]:
#compute count of values in columns
participants['Participant Gender'].value_counts()


Male      40478
Female    24130
Name: Participant Gender, dtype: int64

In [38]:
#compute count of values in columns
participants['Participation Type Name'].value_counts()

Cyclist                                                          22916
Two-Day Participant                                               8875
Cyclist No Bus                                                    3753
$75 Cyclist Registration, No Coach needed on Sunday               2622
Two Day Cyclist                                                   2389
Two-day Cyclist                                                   2108
Event Participant                                                 1960
$60 Cyclist Registration, No Coach needed on Sunday               1652
Participant - No Bus                                              1399
Cyclist w/ Sunday PM Bus                                          1330
Cyclist - No Bus                                                  1228
166 miles                                                          958
Virtual Cyclist                                                    937
Cyclist Registration (no transportation needed on Sunday)          866
$95 Cy

In [39]:
#compute count of values in columns
participants['Is Prior Participant'].value_counts()

No     34177
Yes    31322
Name: Is Prior Participant, dtype: int64

In [40]:
#compute count of values in columns
participants['Participant Email Status'].value_counts()

Good                 57899
Unknown               3047
Bad (Hard Bounce)     2911
Bad (Soft Bounce)     1587
Name: Participant Email Status, dtype: int64

In [41]:
#compute count of values in columns
participants['Participant Employer'].value_counts()


# initial results yielded numerous years, social security numbers, and obvious space filler entries. These were deleted.

# Other data entry errors were identified and corrected, such as "AT&T", "A   T&T" and anyone who indicated retired from a firm was changed to retired.
# If in doubt, nothing was changed

Self Employed                                       1381
Retired                                              864
Salesforce                                           227
Google                                               212
Wells Fargo                                          201
IHS                                                  185
Salesforce.com                                       176
National MS Society                                  166
Deloitte                                             153
Johns Manville                                       149
Intuit                                               139
Student                                              126
None                                                 117
Kaiser Permanente                                    110
Wescom Credit Union                                  109
Chevron                                               91
PwC                                                   89
Southern California Edison Comp

In [42]:
#compute count of values in columns
participants['Participant Occupation'].value_counts()


Healthcare                               2315
Engineering                              2225
Information Technology (IT)              1718
Sales                                    1442
Executive/Management                     1224
Education and Training                   1199
Consulting                                844
Banking and Financial Services            828
Accounting                                814
Legal and Paralegal                       708
Marketing                                 606
Construction and Landscaping              526
Student                                   505
Real Estate, Rental, and Leasing          496
Administrative, Support, and Clerical     475
Science and Biotechnology                 378
Government                                357
Insurance                                 347
Manufacturing                             341
Retail/Wholesale                          336
Skilled Work and Trades                   329
Nonprofit                         

In [43]:
#compute count of values in columns
participants['Address  -  Participant City'].value_counts()






Denver                 3194
San Francisco          2315
San Diego              1300
Littleton              1105
Wilmington              966
Los Angeles             897
Jacksonville            779
Lakewood                693
Fort Collins            626
Centennial              575
Boulder                 570
Arvada                  567
Phoenix                 533
Aurora                  519
Tampa                   494
Oakland                 489
Orlando                 488
San Jose                488
Newark                  487
Highlands Ranch         484
Birmingham              473
Golden                  454
Parker                  424
Irvine                  407
Huntington Beach        362
Las Vegas               352
Longmont                349
Westminster             343
Long Beach              343
Englewood               330
                       ... 
Fort Defiance             1
Puyallup                  1
Mount Laurel              1
Chesapeake                1
Dobbs Ferry         

In [44]:
#describe zip code for errors
participants['Address -  Participant ZIP/Postal Code'].describe()

# the zero evalue entries were all intenational. 

count     65478
unique     5554
top       80127
freq        385
Name: Address -  Participant ZIP/Postal Code, dtype: int64

# Describing Data

In [46]:
participants.describe()

,Fiscal Year,Team ID,Contact ID,Emails Sent,Total of All Confirmed Gifts($),Total From Participant($),Total Not From Participant($),Number From Participant,Number Not From Participant,Event ID,Participant Goal($),Suggested Participant Goal($)
count,65499.000000,53990.00000,6.549900e+04,65499.000000,65499.000000,65499.000000,65499.000000,65499.000000,65499.000000,65499.000000,6.549900e+04,65499.000000
mean,2014.810165,399381.94349,1.043183e+07,32.358143,856.559061,127.000670,748.675758,0.775752,7.686362,24217.106551,1.359181e+03,726.219738
std,1.377881,65332.18553,4.186126e+06,269.046299,2490.747586,492.649481,2403.297757,1.089874,12.260705,3045.772863,8.447520e+04,391.786898
min,2013.000000,266665.00000,1.566055e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17966.000000,0.000000e+00,0.000000
25%,2014.000000,348060.00000,7.681859e+06,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,21507.000000,3.000000e+02,300.000000
50%,2015.000000,401596.00000,1.163842e+07,0.000000,400.000000,35.000000,340.000000,1.000000,4.000000,24022.000000,5.000000e+02,900.000000
75%,2016.000000,456011.00000,1.366256e+07,6.000000,800.000000,150.000000,701.000000,1.000000,10.000000,27037.000000,1.001000e+03,1075.000000
max,2017.000000,534794.00000,1.636498e+07,27354.000000,203791.000000,50093.000000,203791.000000,53.000000,552.000000,29050.000000,2.147484e+07,5000.000000


In [47]:
# counting 1 values

print((participants[['Security Category Name','Fiscal Year','Internal Event Name','Event Date','Participation Type Name','Team Name','Team Creation Date','Team Division','Team ID','Contact ID','Member ID','Participant Accept Email','Registration Date','Registration Active Status','Is Team Captain','Is Secondary Registration','Is Prior Participant','Emails Sent','Total of All Confirmed Gifts($)','Total From Participant($)','Total Not From Participant($)','Number From Participant','Number Not From Participant','Participant Email Status','Participant Employer','Participant Occupation','Participant Connection to MS','Address  -  Participant State/Province','Address  -  Participant County','Address  -  Participant City','Address -  Participant ZIP/Postal Code','Registration Type','Event ID','Participant Gender','Participant Goal($)','Suggested Participant Goal($)','Source Code Type','Source Code Text','Sub Source Code Text']] == 1).sum())


Security Category Name                        0
Fiscal Year                                   0
Internal Event Name                           0
Event Date                                    0
Participation Type Name                       0
Team Name                                     0
Team Creation Date                            0
Team Division                                 0
Team ID                                       0
Contact ID                                    0
Member ID                                     0
Participant Accept Email                  43948
Registration Date                             0
Registration Active Status                    0
Is Team Captain                            6195
Is Secondary Registration                   448
Is Prior Participant                          0
Emails Sent                                3138
Total of All Confirmed Gifts($)               2
Total From Participant($)                     6
Total Not From Participant($)           

# FIRST PRIORITY: CORPORATE ACQUISITION
  * What are the greatest growth opportunities for new corporate teams?
  * Can we apply those opportunities to specific rides/markets, especially our biggest events?
  * What industries have had the strongest involvement in Bike MS in the last five years?
  * What occupations were responsible for most of our fundraising?
  * Can we tie together these industries and occupations to identify gaps/opportunities?
  * What is the common denominator for our top performing corporate teams? (Is it industry, culture, executive involvement, connection to MS, other?)
  * Can we quantify the effect competing events are having in our top markets? 
Top competitors:
   1. Best Buddies
     * HYANNIS PORT June 2, 2018 Boston to Hyannis Port, MA
     * HEARST CASTLE September 8, 2018 San Simeon, CA
     * MIAMI November 16, 2018 Miami, FL
   2. Peddle the Cause
     * Louis
     * San Diego
   3. Pelatonia – OH
   4. PanMass Challenge - Boston
   5. ADA – Tour de Cure
   6. Jude Children’s Research Hospital
   7. JDRF Ride to Cure Diabetes
   8. Cycle for Life - Cystic Fibrosis Foundation
   9. Ride to Recovery
   10. Virtual Cycling
   11. Athletic Club Stationary Cycling Relays
   12. American Cancer Society
   13. Leukemia and Lymphoma Society – Team in Training

# SECOND PRIORITY: DIGITAL/SOCIAL ACQUISITION
  * What are the greatest opportunities for digital marketing investments? Where have we seen the greatest ROI?
  * Once someone is registered, what tactics and behaviors drive fundraising, and at what times leading up to the event?
  * What behavioral data do you see about usage of our fundraising tools and how it may or may not relate to performance of top fundraisers?
  * Despite increasing our digital advertising spend, acquisition continues to trend downward overall. Why? Is it an issue of needing more traffic, better targeting, or a conversion rate issue that needs to be addressed through the registration process? What can we do to reverse the trend?


# FINALLY:
  * As you studied this data, is there something else that came up as an insight into our operations that the questions above do not capture? 


# Descriptive Analytics